In [38]:
import pandas as pd 
import openai
import requests
import json
import re
from collections import defaultdict


Documentation

This code processes a dataset named jameel_ds to clean and organize its contents:

- Extracting Duplicated Rows:
    It first identifies and stores any duplicated rows in a separate variable, duplicated_rows.

- Removing Duplicate Entries:
    The code then removes duplicate entries based on the 'Description' column. This means if there are multiple rows with the same description, only the first one is kept.

- Dropping Unnecessary Columns:
    It removes an irrelevant column (named 'Field1') from the dataset.

- Resetting the Index:
    The index of the DataFrame is reset, making it sequential and more organized, especially after the removal of some rows.

- Renaming Columns:
    Column names are changed to more descriptive titles (e.g., 'Text' is renamed to 'Part Name', 'Text1' to 'Part Number', etc.), making the dataset easier to understand.

- Cleaning 'Part Number' Column:
    Any leading or trailing spaces in the 'Part Number' column are removed for consistency. Additionally, a specific text pattern ('رقم الصنف' followed by any whitespace) is removed from this column to make the part numbers cleaner.

- Cleaning 'Price (SAR)' Column:
    In the 'Price (SAR)' column, the text pattern 'ريال للقطعة', along with any surrounding whitespace, is removed. This likely transforms the entries in this column to simple numerical values, indicating the price.



In [87]:
jameel_ds = pd.read_csv("/home/qparts/Desktop/Jameel Toyota project /Toyota_jameel(1).csv",on_bad_lines='skip')
duplicates = jameel_ds.duplicated(keep=False)
num_duplicates = duplicates.sum()

# Displaying duplicated rows (optional)
duplicated_rows = jameel_ds[duplicates]
cleaned_jameel = jameel_ds.drop_duplicates(subset='Text3', keep='first')
cleaned_jameel = cleaned_jameel.drop(columns=['Field1'])
cleaned_jameel = cleaned_jameel.reset_index(drop=True)
cleaned_jameel = cleaned_jameel.rename(columns={'Text':'Part Name', 'Text1':'Part Number', 'Text2':'Dimensions', 'Text3':'Description', 'Field5':'Category', 'Field6':'Sub-category', 'Field7':'Price (SAR)'})
cleaned_jameel['Part Number'] = cleaned_jameel['Part Number'].str.strip()
cleaned_jameel['Part Number'] = cleaned_jameel['Part Number'].str.replace(r'رقم الصنف\s*', '', regex=True)
cleaned_jameel['Price (SAR)'] = cleaned_jameel['Price (SAR)'].str.replace(r'\s*ريال\s*للقطعة', '', regex=True)
print(cleaned_jameel['Description'].count(),"*****")
cleaned_jameel.head(5)

305 *****


,Part Name,Part Number,Dimensions,Description,Category,Sub-category,Price (SAR)
0,طوق اللوحة الخلفية لرأس الاسطوانة,1118235050,\n \n ...,\n هذه القطعة مناسبة للموديلات ال...,NaN,\n \n,27.00
1,خزان الرادياتير الاحتياطي,1647031050,\n \n ...,\n هذه القطعة مناسبة للموديلات ال...,NaN,\n \n,205.50
2,معدات كوب أسطوانة العجلة الخلفية,0490636060,\n \n ...,\n هذه القطعة مناسبة للموديلات ال...,NaN,\n \n,106.50
3,"عمود التوازن عمود التوازن , رقم 2",1362275020,\n \n ...,\n هذه القطعة مناسبة للموديلات ال...,NaN,\n \n,214.20
4,المبرد زيت,1571017021,\n \n ...,\n هذه القطعة مناسبة للموديلات ال...,التبريد والتسخين,\n التبريد والتسخين / تبريد / م...,"1,588.80"


# Vehicle Information Extraction and Organization Process

This code is designed to extract and organize specific information from text data, particularly focused on vehicle models and their variants. The process includes the following steps:

## 1. Text Preparation and Segmentation
- Breaks down a large text into smaller chunks to manage text size limits of the processing API.
- Uses a specific delimiter and a token limit to ensure manageable sizes of text chunks while maintaining meaningful text segments.

## 2. Conversation Handling with an AI Model
- Initiates a conversation with an AI model (like GPT-3.5 or GPT-4) for each text chunk.
- Tasks the AI to extract information in a structured JSON format, including vehicle model, production start and end dates, and variants.

## 3. Response Processing and Error Handling
- Collects and checks responses from the AI for errors.
- Processes valid responses further; displays error messages if responses are not valid.

## 4. Data Extraction and Transformation
- Attempts to parse the AI's response into a structured format (JSON).
- Transforms this JSON data into a tabular format, creating entries for each vehicle model and its variants.

## 5. Data Aggregation and Final Formatting
- Further aggregates and organizes the extracted data. Combines variants of each model into a single entry for clarity and conciseness.
- Creates a DataFrame from this aggregated data for a structured and tabular presentation.

## 6. Output Generation
- Saves the DataFrame to an Excel file, providing a structured and easy-to-read document.
- Prints the DataFrame for a quick review of the processed data.

**Purpose**: The code automates the extraction of detailed vehicle information from text, processes and structures this information, and presents it in an organized, tabular format suitable for further analysis or reporting.


In [37]:

GPT_MODEL = "gpt-3.5-turbo-0613"
# GPT_MODEL = "gpt-4-0613"
openai.api_key =""





def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "temperature": 0.0 ,"messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def execute_function_call(message):
    try:
        # Attempt to parse the JSON data
        data = json.loads(message["function_call"]["arguments"])

        # Print the data for debugging
        print("Function call returned data:", data)

        # Access the required_data_json_format key
        if "required_data_json_format" in data:
            info = data["required_data_json_format"]
            return info
        else:
            return "Key 'required_data_json_format' not found in the function call response."

    except json.JSONDecodeError as e:
        return f"JSON decoding error: {e}"
    except KeyError as e:
        return f"Key error: {e}"


messages = []
# def run_conversation(user_input):
#     # messages.append({"role": "system", "content": "Extract the following information from the next user text in JSON format: 'Model', 'Production Start', 'Production End', 'Variant', 'Features', 'Regions', and 'Additional Info'.from the provided text"})
    
#     user_message = ({"role": "user", "content":f"Extract the following information from text in JSON format: 'Model', 'Production Start', 'Production End', 'Variant', 'Features', 'Regions', and 'Additional Info'.from the provided text {user_input}"})
#     messages_to_send = [user_message]

#     chat_response = chat_completion_request(messages_to_send)
#     if chat_response.status_code != 200:
#         print(f"Error: Unable to get a response from OpenAI. Status code: {chat_response.status_code}, Response: {chat_response.content.decode('utf-8')}")
#     assistant_message = chat_response.json()["choices"][0]["message"]
#     # messages.append(assistant_message)
#     if assistant_message.get("function_call"):
#         info_ext = execute_function_call(assistant_message)
#         return info_ext 
def chunk_text_by_token_limit_and_delimiter(text):
    # Specify the maximum number of tokens per chunk
    max_tokens = 2000  # Adjust as needed

    # Define the delimiter
    delimiter = "من سنةحتى سنةملاحظات على الموديل"
    # Approximate character count per token
    chars_per_token = 4

    # Calculate max characters per chunk based on token limit
    max_chars = max_tokens * chars_per_token

    # Split the text using the specified delimiter
    parts = re.split(delimiter, text)
    chunks = []
    current_chunk = ""

    for part in parts:
        # Check if adding this part exceeds the max character limit
        if len(current_chunk) + len(part) > max_chars and current_chunk:
            # If it does, add the current chunk to the list and start a new one
            chunks.append(current_chunk)
            current_chunk = part
        else:
            # If it doesn't, add the part to the current chunk
            current_chunk += (delimiter if current_chunk else "") + part

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(current_chunk)

    return chunks



def run_conversation(user_input):
    # system_message = {"role": "system", "content": "Extract the following information from the next user text in standard JSON format: 'Model', 'Production Start', 'Production End', and 'Variants'.from the provided text"}
    
    # Prepare the message in the format expected by the OpenAI API
    user_message = {
    "role": "user",
    "content": f'''Please provide the extracted information from this text {user_input}in a flattened JSON format. Each entry should adhere to the following structure:

            - 'Model': The name or identifier of the vehicle model.
            - 'Production Start': The date when the production of the model started, formatted as YYYY/MM.
            - 'Production End': The date when the production of the model ended, formatted as YYYY/MM. If the model is still in production, fill this with 'NA'.
            - 'Variants': A list containing only the 'Code' for each variant of the model.

            Each variant should be listed as a separate entry. If the 'Production End' is unknown or if the model is still in production, please use 'NA' for that field. For variants, only the unique code that identifies each variant is required, without the need for a description of the specifications or features.
                '''
            }
    messages_to_send = [user_message]

    # Assuming 'chat_completion_request' is a function that makes the API call
    chat_response = chat_completion_request(messages_to_send)
    
    if chat_response.status_code != 200:
        return {
            "error": f"Unable to get a response from OpenAI. Status code: {chat_response.status_code}",
            "response": chat_response.content.decode('utf-8')
        }

    # Parse the assistant's message from the response
    assistant_message = chat_response.json()["choices"][0]["message"]["content"]
    # print('Assistant :' , assistant_message)
    
    # Assuming the assistant_message is a string that contains the JSON-formatted information
    # Convert the JSON string back to a dictionary (if it's not already a dictionary)
    try:
        info_ext = json.loads(assistant_message)
        return info_ext
    except json.JSONDecodeError:
        print("Assistant :",assistant_message)
        return {"error": "Failed to parse the assistant's response as JSON."}
        

    


def convert_json_to_dataframe(json_str):
    # Parse the JSON string into a Python object (list of dictionaries)
    data = json.loads(json_str)

    # Create a DataFrame from the list of dictionaries
    return pd.DataFrame(data)
def reduce_spaces(text):
    # Remove extra spaces and line breaks
    text = re.sub(r'\s+', ' ', text).strip()
    return text






text_chunks = chunk_text_by_token_limit_and_delimiter(text)
messages = []
combined_responses = []
for chunk in text_chunks:
    response = run_conversation(chunk)
        # Check if response is not empty and the first item is not an error
     # Check if the response is a list and is not empty
    if isinstance(response, list) and response:
        # Check if the first item in the list contains an error key
        if isinstance(response[0], dict) and "error" not in response[0]:
            combined_responses.extend(response)
        else:
            print("Error in chunk response:", response[0])
    else:
        print("Unexpected response format or empty response:", response)




# reduced_text  = reduce_spaces(text)
# json_output = run_conversation(reduced_text)
# print(json_output)

flattened_data = []
for response in combined_responses:
    if isinstance(response, dict):
        entries = [response]
    elif isinstance(response, list):
        entries = response
    else:
        print("Unexpected response format. Skipping this response.")
        continue

    for entry in entries:
        if isinstance(entry, dict):
            model = entry.get('Model', 'NA')
            prod_start = entry.get('Production Start', 'NA')
            prod_end = entry.get('Production End', 'NA')
            variants = entry.get('Variants', [])

            if all(isinstance(variant, str) for variant in variants):
                for variant in variants:
                    variant_data = {
                        'Model': model,
                        'Production Start': prod_start,
                        'Production End': prod_end,
                        'Variant Code': variant
                    }
                    flattened_data.append(variant_data)
            else:
                print(f"Variants for model {model} are not in the expected string format.")
        else:
            print("Entry is not a dictionary. Skipping this entry.")

# Group and combine data
grouped_data = defaultdict(set)

for entry in flattened_data:
    key = (entry['Model'], entry['Production Start'], entry['Production End'])
    grouped_data[key].add(entry['Variant Code'])

combined_data = []

for (model, prod_start, prod_end), variants in grouped_data.items():
    combined_data.append({
        'Model': model,
        'Production Start': prod_start,
        'Production End': prod_end,
        'Variant Codes': '; '.join(sorted(variants))
    })
# Create DataFrame
df = pd.DataFrame(combined_data)


# Save to Excel
df.to_excel("vehicle_info.xlsx", index=False)

# Print DataFrame
print(df)

print("********************************************************************")

                   Model Production Start Production End  \
0               FORTUNER          2005/01        2012/04   
1               FORTUNER          2011/07        2021/04   
2               FORTUNER          2015/07        2021/04   
3               FORTUNER          2020/06        2022/05   
4         FORTUNER (SUV)          2021/08        2022/10   
5         FORTUNER (SUV)          2022/08             NA   
6                  HIACE          2005/08        2007/09   
7                  HIACE          2005/08             NA   
8                  HIACE          2005/12        2007/09   
9                  HILUX           200408         201204   
10                 HILUX           201107         202110   
11                 HILUX           201505         202105   
12                 HILUX           202006         202206   
13                 HILUX           202006         202202   
14    HILUX (DOUBLE CAB)           202108             NA   
15    HILUX (DOUBLE CAB)           20220